In [144]:
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

In [145]:
from sqlalchemy import create_engine
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import yaml
py.offline.init_notebook_mode(connected=True)

In [146]:
model = 'mlp_1'
strategy = 'value_bet_0.2'
print("Model: {model}, Strategy: {strategy}".format(model=model, strategy=strategy))

Model: mlp_1, Strategy: value_bet_0.2


In [147]:
def read_yaml(yaml_file):
    with open(yaml_file, 'r') as ymlfile:
        yml = yaml.load(ymlfile)
    return yml
config_file = '../configs/mlp1_config.yml'
cfg = read_yaml(config_file)

In [148]:
db = create_engine("mysql://root@localhost/football_data")
df = pd.read_sql(sql="""
select c.MATCH_ID, c.Date, c.competition_code, c.season_code, 
h.long_name as HomeTeam, aw.long_name as AwayTeam,
1/a.pH as fH, 1/a.pD as fD, 1/a.pA as fA,
c.BbAvH, c.BbAvD, c.BbAvA,
bH, bD, bA, payout, FTR
from match_prob a 
join match_bet b on a.MODEL = b.MODEL and a.MATCH_ID=b.MATCH_ID
join match_teams c on a.MATCH_ID = c.MATCH_ID
join teams h on c.HomeTeam = h.team_id 
join teams aw on c.AwayTeam = aw.team_id
where b.MODEL = '{model}' and b.STRATEGY='{strategy}'""".format(model=model, strategy=strategy),
con=db)

In [149]:
sum(df.payout)

145.71000000000009

In [150]:
def plot_timeseries(df, att, title):
    timeseries = []
    for i in att:
        sub_df = df.groupby(['Date'], as_index=True).sum()[[i]]
        trace = go.Scatter(
            x = sub_df.index,
            y = sub_df[i],
            mode = 'lines',
            name = i
        )
        timeseries.append(trace)
        lyt = go.Layout(
            title = title
        )
    fig = go.Figure(data=timeseries, layout=lyt)
    py.offline.iplot(fig)
#plot_timeseries(df, ['payout'], 'payout')

In [151]:
df_train = df.loc[df.season_code.isin(cfg['train_seasons'])]
df_test = df.loc[df.season_code.isin(cfg['test_seasons'])]

In [152]:
train_cum_payout = df_train[['Date', 'payout']].groupby('Date').sum().cumsum(0).reset_index()
test_cum_payout = df_test[['Date', 'payout']].groupby('Date').sum().cumsum(0).reset_index()

In [153]:
plot_timeseries(train_cum_payout, ['payout'], 'cum_payout')
plot_timeseries(test_cum_payout, ['payout'], 'cum_payout')

In [154]:
def bar_plot(df, attr):
    sub_df = df[[attr, 'payout']].groupby(attr).sum()
    data = [go.Bar(
                x=sub_df.index,
                y=sub_df.payout
        )]
    py.offline.iplot(data, filename='basic-bar')

In [155]:
df_train['month'] = pd.DatetimeIndex(df_train['Date']).month
df_test['month'] = pd.DatetimeIndex(df_test['Date']).month
bar_plot(df_train, 'month')
bar_plot(df_test, 'month')

C:\Users\pfinardi\AppData\Local\Continuum\Anaconda3\envs\bma\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Users\pfinardi\AppData\Local\Continuum\Anaconda3\envs\bma\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [156]:
bar_plot(df_train, 'competition_code')
bar_plot(df_test, 'competition_code')